In [2]:
!pip install pandas
import pandas as pd
!pip install numpy
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm, datasets
!pip install matplotlib
import matplotlib.pyplot as plt
import nibabel as nib
import keras
from keras.layers import Input
from tensorflow.keras.applications import ResNet50
!pip install scikit-image
import skimage.transform as skTrans

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


2023-10-23 16:44:00.103944: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 16:44:00.547760: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-10-23 16:44:00.547806: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-10-23 16:44:00.547811: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [3]:
def path_data(category,patient):
    path = "./data/images/GM/"+category+"/axial/"+patient+".png"
    return path

In [4]:
#reading datas

total_data = pd.read_csv("./total_data.csv")
total_data = total_data[(total_data["Research Group"] != "MCI")]
total_data = total_data.reset_index()
PID = total_data["Subject ID"]
Labels = total_data["Research Group"]


In [5]:
paths = []
for i in range(0,len(PID)):
    paths.append(path_data(Labels[i],PID[i]))
    
paths

['./data/images_gray/GM/CN/axial/002_S_0295.png',
 './data/images_gray/GM/CN/axial/002_S_0413.png',
 './data/images_gray/GM/CN/axial/002_S_0559.png',
 './data/images_gray/GM/AD/axial/002_S_0619.png',
 './data/images_gray/GM/CN/axial/002_S_0685.png',
 './data/images_gray/GM/AD/axial/002_S_0816.png',
 './data/images_gray/GM/AD/axial/002_S_0938.png',
 './data/images_gray/GM/AD/axial/002_S_0955.png',
 './data/images_gray/GM/AD/axial/002_S_1018.png',
 './data/images_gray/GM/CN/axial/002_S_1261.png',
 './data/images_gray/GM/CN/axial/002_S_1280.png',
 './data/images_gray/GM/CN/axial/002_S_4213.png',
 './data/images_gray/GM/CN/axial/002_S_4225.png',
 './data/images_gray/GM/CN/axial/002_S_4262.png',
 './data/images_gray/GM/CN/axial/002_S_4264.png',
 './data/images_gray/GM/CN/axial/002_S_4270.png',
 './data/images_gray/GM/AD/axial/002_S_5018.png',
 './data/images_gray/GM/CN/axial/002_S_6053.png',
 './data/images_gray/GM/CN/axial/003_S_0907.png',
 './data/images_gray/GM/CN/axial/003_S_0981.png',


In [6]:
new_labels = []
for i in Labels:
    if i == "CN":
        new_labels.append(0);
    elif i == "MCI":
        new_labels.append(2);
    else:
        new_labels.append(1); 

In [7]:
data = []# List of tuples (image_path, label)
for i in range(0,len(paths)):
    data.append((paths[i],new_labels[i]))

data

[('./data/images_gray/GM/CN/axial/002_S_0295.png', 0),
 ('./data/images_gray/GM/CN/axial/002_S_0413.png', 0),
 ('./data/images_gray/GM/CN/axial/002_S_0559.png', 0),
 ('./data/images_gray/GM/AD/axial/002_S_0619.png', 1),
 ('./data/images_gray/GM/CN/axial/002_S_0685.png', 0),
 ('./data/images_gray/GM/AD/axial/002_S_0816.png', 1),
 ('./data/images_gray/GM/AD/axial/002_S_0938.png', 1),
 ('./data/images_gray/GM/AD/axial/002_S_0955.png', 1),
 ('./data/images_gray/GM/AD/axial/002_S_1018.png', 1),
 ('./data/images_gray/GM/CN/axial/002_S_1261.png', 0),
 ('./data/images_gray/GM/CN/axial/002_S_1280.png', 0),
 ('./data/images_gray/GM/CN/axial/002_S_4213.png', 0),
 ('./data/images_gray/GM/CN/axial/002_S_4225.png', 0),
 ('./data/images_gray/GM/CN/axial/002_S_4262.png', 0),
 ('./data/images_gray/GM/CN/axial/002_S_4264.png', 0),
 ('./data/images_gray/GM/CN/axial/002_S_4270.png', 0),
 ('./data/images_gray/GM/AD/axial/002_S_5018.png', 1),
 ('./data/images_gray/GM/CN/axial/002_S_6053.png', 0),
 ('./data/

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from PIL import Image
from torch.optim import lr_scheduler

# Define the ResNet-18 model
model = models.resnet18(pretrained=True)
model
# model

/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
# Define the data loader and transformations
class ImageDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path, label = self.data[index]
#         img = Image.open(img_path)
        img = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [10]:
transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision.transforms as transforms
# from torchvision.datasets import ImageFolder
# from torch.utils.data import DataLoader, SubsetRandomSampler
# from sklearn.model_selection import KFold
# from torchvision.models import resnet18

# # Define the DeepSVM model with ResNet18 as the base
# class DeepSVM_ResNet(nn.Module):
#     def __init__(self):
#         super(DeepSVM_ResNet, self).__init__()
#         self.resnet = resnet18(pretrained=True)
#         self.svm = nn.Linear(1000, 3)

#     def forward(self, x):
#         x = self.resnet(x)
#         return self.svm(x)

# # Define the loss function
# def hinge_loss(scores, targets):
#     margin = 1 - scores * targets
#     return torch.mean(torch.max(torch.zeros(margin.size()), margin))

# # Define the training function
# def train(model, optimizer, x, y):
# #     x = x.to(device)
# #     y = y.to(device)
#     optimizer.zero_grad()
#     scores = model(x)
#     loss = hinge_loss(scores.view(-1), y)
#     loss.backward()
#     optimizer.step()
#     return loss.item()

In [12]:
class ImageDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path, label = self.data[index]
#         img = Image.open(img_path)
        img = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [13]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [14]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision.transforms as transforms
# from torchvision.datasets import ImageFolder
# from torch.utils.data import DataLoader, SubsetRandomSampler
# from sklearn.model_selection import KFold
# from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
# from torchvision.models import resnet18

# # Define the DeepSVM model with ResNet18 as the base
# class DeepSVM_ResNet(nn.Module):
#     def __init__(self):
#         super(DeepSVM_ResNet, self).__init__()
#         self.resnet = resnet18(pretrained=True)
#         self.svm = nn.Linear(1000, 3)

#     def forward(self, x):
#         x = self.resnet(x)
#         return self.svm(x)

# # Define the loss function
# def hinge_loss(scores, targets):
#     margin = 1 - scores * targets
#     return torch.mean(torch.max(torch.zeros(margin.size()), margin))

# # Define the training function
# def train(model, optimizer, x, y):
#     x = x.to(device)
#     y = y.to(device)
#     optimizer.zero_grad()
#     scores = model(x)
#     loss = hinge_loss(scores.view(-1), y)
#     loss.backward()
#     optimizer.step()
#     return loss.item()

# # Set device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Load data
# # data_dir = 'path/to/data'
# # dataset = ImageFolder(data_dir, transform=transforms.Compose([
# #     transforms.Resize((256, 256)),
# #     transforms.ToTensor(),
# # ]))
# # labels = [0] * len(dataset)
# kf = KFold(n_splits=5, shuffle=True)
# train_losses = []
# test_losses = []
# train_accs = []
# test_accs = []
# train_f1s = []
# test_f1s = []
# criterion = nn.CrossEntropyLoss()
# for fold, (train_idx, test_idx) in enumerate(kf.split(data)):
#     print(f'Fold {fold+1}')
#     train_data = [data[i] for i in train_idx]
#     val_data = [data[i] for i in test_idx]
# #     train_sampler = SubsetRandomSampler(train_idx)
# #     test_sampler = SubsetRandomSampler(test_idx)
# #     train_loader = DataLoader(dataset, batch_size=64, sampler=train_sampler)
# #     test_loader = DataLoader(dataset, batch_size=64, sampler=test_sampler)
#     train_dataset = ImageDataset(train_data, transform=transform)
#     val_dataset = ImageDataset(val_data, transform=transform)
#     train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

#     # Initialize the model and optimizer
#     model = DeepSVM_ResNet().to(device)
#     optimizer = optim.SGD(model.parameters(), lr=0.1)

#     # Train the model
#     for epoch in range(10):
#         train_loss = 0
#         train_preds = []
#         train_targets = []
#         for images, labels in train_loader:
# #             loss = train(model, optimizer, images, labels)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             train_loss += loss.item()
#             optimizer.step()
#             train_preds.extend(output.argmax(dim=1).tolist())
#             train_targets.extend(labels.tolist())
# #         for batch_idx, (data, target) in enumerate(train_loader):
# #             loss = train(model, optimizer, data, target)
# #             train_loss += loss
# #             train_preds.extend(model(data).argmax(dim=1).tolist())
# #             train_targets.extend(target.tolist())
#         train_loss /= len(train_loader)
#         train_losses.append(train_loss)
#         train_acc = accuracy_score(train_targets, train_preds)
#         train_f1 = f1_score(train_targets, train_preds, average='weighted')
#         train_accs.append(train_acc)
#         train_f1s.append(train_f1)
#         print(f'Epoch: {epoch}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}')

#     # Evaluate the model on test data
#     test_loss = 0
#     test_preds = []
#     test_targets = []
#     with torch.no_grad():
#         for images, labels in val_loader:
# #             images = images.to(device)
# #             labels = labels.to(device)
# #             scores = model(images)
# #             loss = hinge_loss(scores.view(-1), labels)
# #             test_loss += loss.item() * images.size(0)
# #             pred = scores.argmax(dim=1)
# #             test_preds.extend(pred.tolist())
# #             test_targets.extend(target.tolist())
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             test_loss += loss.item()
#             optimizer.step()
#             test_preds.extend(output.argmax(dim=1).tolist())
#             test_targets.extend(labels.tolist())
# #         for data, target in test_loader:
# #             data = data.to(device)
# #             target = target.to(device)
# #             scores = model(data)
# #             loss = hinge_loss(scores.view(-1), target)
# #             test_loss += loss.item() * data.size(0)
# #             pred = scores.argmax(dim=1)
# #             test_preds.extend(pred.tolist())
# #             test_targets.extend(target.tolist())
#         test_loss /= len(test_loader.dataset)
#         test_losses.append(test_loss)
#         test_acc = accuracy_score(test_targets, test_preds)
#         test_f1 = f1_score(test_targets, test_preds, average='weighted')
#         test_accs.append(test_acc)
#         test_f1s.append(test_f1)
#         print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}, Test F1: {test_f1:.4f}')

#     # Print confusion matrix
#     cm = confusion_matrix(test_targets, test_preds)
#     print(f'Confusion Matrix:\n{cm}')

# print(f'Average Train Loss: {sum(train_losses)/len(train_losses):.4f}')
# print(f'Average Test Loss: {sum(test_losses)/len(test_losses):.4f}')
# print(f'Average Train Accuracy: {sum(train_accs)/len(train_accs):.4f}')
# print(f'Average Test Accuracy: {sum(test_accs)/len(test_accs):.4f}')
# print(f'Average Train F1 Score: {sum(train_f1s)/len(train_f1s):.4f}')
# print(f'Average Test F1 Score: {sum(test_f1s)/len(test_f1s):.4f}')

In [15]:
# # Load the data and split into k folds
# k = 5
# kf = KFold(n_splits=k, shuffle=True)
# from torch.nn import functional as F


# # Define the loss function and optimizer
# # criterion = nn.CrossEntropyLoss()
# criterion = nn.MultiLabelMarginLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# # optimizer = optim.SGD(model.parameters(), momentum=0.99, lr=0.001)

# # Define the learning rate scheduler
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.9)

# for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
#     train_data = [data[i] for i in train_idx]
#     val_data = [data[i] for i in val_idx]
    
#     # Define the data loaders
#     train_dataset = ImageDataset(train_data, transform=transform)
#     val_dataset = ImageDataset(val_data, transform=transform)
#     train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
    
#     # Define the model
#     model = models.resnet18(pretrained=True)
#     num_ftrs = model.fc.in_features
#     for param in model.parameters():
#         param.requires_grad = False
#     model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 3),
# #                               nn.BatchNorm1d(128),
# #                               nn.ReLU(inplace=True),
# #                               nn.Dropout(p=0.1),
# #                               nn.Linear(512,128),
# # #                               nn.BatchNorm1d(3),
# #                               nn.ReLU(inplace=True),
# # #                               nn.Dropout(p=0.1),
# #                               nn.Linear(128,3),
# #                               nn.ReLU(inplace=True),
# #                               nn.Dropout(p=0.1),
#                               nn.Softmax(dim=1))

#     # Train the model
#     for epoch in range(30):
#         model.train()
#         train_loss = 0
#         train_correct = 0
#         train_f1 = 0
#         for images, labels in train_loader:
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
# #             torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
# #             torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#             optimizer.step()
#             train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             train_correct += (predicted == labels).sum().item()
#             train_f1 += f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')

#         # Evaluate the model on the validation set
#         model.eval()
#         with torch.no_grad():
#             val_loss = 0
#             val_correct = 0
#             val_f1 = 0
#             for images, labels in val_loader:
#                 outputs = model(images)
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.item()
#                 _, predicted = torch.max(outputs.data, 1)
#                 val_correct += (predicted == labels).sum().item()
#                 val_f1 += f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')
#             train_acc = 100 * train_correct / len(train_dataset)
#             train_f1 = 100 * train_f1 / len(train_loader)
#             val_acc = 100 * val_correct / len(val_dataset)
#             val_f1 = 100 * val_f1 / len(val_loader)
#             print(f'Fold {fold+1}, Epoch {epoch+1}, Train Accuracy: {train_acc:.2f}%, Train F1 Score: {train_f1:.2f}%, Train Loss: {train_loss:.4f}, Validation Accuracy: {val_acc:.2f}%, Validation F1 Score: {val_f1:.2f}%, Validation Loss: {val_loss:.4f}')

#         # Update the learning rate
#         exp_lr_scheduler.step()

In [16]:
import torch
import torchvision
from torchvision import transforms
from sklearn import svm
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np


# Define the SVM classifier and its hyperparameters
clf = svm.SVC(kernel='linear', C=1, gamma='auto',decision_function_shape='ovo')


k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True)

# Train the model using k-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(data,new_labels)):
    train_data = [data[i] for i in train_idx]
    val_data = [data[j] for j in val_idx]
    
    # Define the data loaders
    train_dataset = ImageDataset(train_data, transform=transform)
    val_dataset = ImageDataset(val_data, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)
    
    # Define the model
    model = torchvision.models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    for param in model.parameters():
        param.requires_grad = False
    model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 150),
                              nn.ReLU(inplace=True))

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
#     optimizer = optim.SGD(model.parameters(), momentum=0.9, lr=0.001)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model on the training set
    # Extract the 128 features from the model for each image in the dataset
    train_features = []
    train_labels = []
    for epoch in range(30):
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            loss.backward()
            train_features.append(outputs.detach().numpy())
            train_labels.append(labels.numpy())
            optimizer.step()
#         print(outputs.shape)
#         print(len(train_features))
    train_features = np.array(train_features)
    train_features.shape
    train_labels = np.array(train_labels)
    n,x,y = train_features.shape
    train_features = train_features.reshape(n*x,y)
#     print(train_features.shape)
#     print(f"label:{train_features.shape}")
    n,x = train_labels.shape
    train_labels = train_labels.reshape(n*x)
#     print(train_labels.shape)

    # Train the SVM classifier on the 128-dimensional feature vectors
    clf.fit(train_features, train_labels)

    # Evaluate the model on the training set
#     train_features = []
#     train_labels = []
#     for images, labels in train_loader:
#         outputs = model(images)
#         train_features.append(outputs.detach().numpy())
#         train_labels.eappend(labels.numpy())
#     train_features = np.array(train_features)
#     train_labels = np.array(train_labels)
    train_acc = clf.score(train_features, train_labels)
    train_f1 = f1_score(train_labels, clf.predict(train_features), average='macro')
    print(f'Fold {fold+1}, Train Accuracy: {train_acc:.2f}%, Train F1 Score: {train_f1:.2f}%')

    # Evaluate the model on the test set
    test_features = []
    test_labels = []
    for images, labels in val_loader:
        outputs = model(images)
        test_features.extend(outputs.detach().numpy())
        test_labels.extend(labels.numpy())
#     print(test_features)
    test_features = np.array(test_features)
    test_labels = np.array(test_labels)
#     print(test_features.shape)
#     test_features = test_features.reshape(n*x,y)
#     print(test_features.shape)
#     print(f"label:{test_features.shape}")
#     n,x = test_labels.shape
#     test_labels = test_labels.reshape(n*x)
#     print(test_labels.shape)
    test_acc = clf.score(test_features, test_labels)
    test_f1 = f1_score(test_labels, clf.predict(test_features), average='macro')
    print(f'Fold {fold+1}, Test Accuracy: {test_acc:.2f}%, Test F1 Score: {test_f1:.2}')

/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1, Train Accuracy: 0.54%, Train F1 Score: 0.52%
Fold 1, Test Accuracy: 0.50%, Test F1 Score: 0.37


/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
test_features.shape

In [ ]:
# np.array(test_features[1:6]).shape

In [ ]:
# outputs.detach().numpy()

In [ ]:
# train_features[1]

In [ ]:
# a = 0
# for images,labels in train_loader:
#     a+=1
    
# print(a)

In [ ]:
# print("hi")